In [3]:
import pandas as pd
from pmdarima import auto_arima

In [5]:
#cargar datos
#importamos a un dataframe

df = pd.read_csv(r"C:\Users\ninic\OneDrive - Lambton College\Term 1\BDM 1113-NoSQL data Base\project\Project-Data-Analysis-Lambton\Electricity_Generation.csv", header=None, names=range(47))
#nombre de las tablas
table_names = ["Canada", "Newfoundland and Labrador", "Prince Edward Island", "Nova Scotia", "New Brunswick","Quebec", "Ontario","Manitoba", "Alberta", "British Columbia", "Saskatchewan", "Yukon", "Northwest Territories", "Nunavut"]
# Identificar grupos basados en los nombres de las tablas
groups = df[0].isin(table_names).cumsum()
# Crear un diccionario de DataFrames para cada tabla
tables = {g.iloc[0,0]: g.iloc[1:-1, :20] for k,g in df.groupby(groups)} 


#acomodar datos
#selecciono una sola region 
df2 = tables["Canada"]
#eliminar primera fila porque los anos estaban en esa fila
df2 = df2.iloc[1:, :]
#eliminar esa columna
df2.drop(df2.columns[0], axis=1, inplace=True)
# Renombrar las columnas del 2005 al 2023
nuevos_nombres = {str(anio): f'Titulo_{anio}' for anio in range(2005, 2024)}
# Asignar nuevos nombres a las columnas
df2.columns = nuevos_nombres
#reset index
df2 = df2.reset_index(drop=True)
#nuevos nombres index
ind_nombres = ["Hydro / Wave / Tidal", "Wind", "Biomass / Geothermal","Solar","Uranium","Coal & Coke","Natural Gas","Oil"]  
df2.rename(index=dict(enumerate(ind_nombres)), inplace=True)
#transponer matriz
dataset = df2.T
#cambiar los numeros a fecha
dataset.index = pd.to_datetime(dataset.index, format='%Y')
#seleccionamos que solo sea WIND y lo pasamos de serie a dataframe 
dataset=dataset["Wind"].to_frame()
print(type(dataset))
print(dataset)


<class 'pandas.core.frame.DataFrame'>
                Wind
2005-01-01   1453.41
2006-01-01   2529.41
2007-01-01   3683.41
2008-01-01   4715.44
2009-01-01   7031.23
2010-01-01   8354.23
2011-01-01  11622.99
2012-01-01  13716.78
2013-01-01  17544.74
2014-01-01  21484.50
2015-01-01  26692.57
2016-01-01  30624.00
2017-01-01  31197.00
2018-01-01  32814.01
2019-01-01  32333.00
2020-01-01  37454.07
2021-01-01  35996.13
2022-01-01  50816.97
2023-01-01  54408.92


C:\Users\ninic\AppData\Local\Temp\ipykernel_47324\1122791778.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2.columns[0], axis=1, inplace=True)


In [12]:



# Ahora, 'datos' es un DataFrame de pandas con tus datos de serie temporal.
# Puedes ajustar el modelo auto_arima directamente con la columna de observaciones.

# Ajusta un modelo auto_arima a tu serie temporal
modelo = auto_arima(dataset['Wind'], seasonal=False, stepwise=True, suppress_warnings=True, error_action="ignore", n_jobs=-1)

# Muestra un resumen del modelo
print(modelo.summary())
#-------------------------------------------

# Hacer un pronóstico para los próximos 4 años (2024 a 2027)
pronostico_2024_a_2027 = modelo.predict(n_periods=4)
print(pronostico_2024_a_2027)
# Los años para los cuales se realiza el pronóstico
años = [2024, 2025, 2026, 2027]

# Crear un DataFrame para mostrar los pronósticos de forma más amigable
pronosticos_df = pd.DataFrame(data=pronostico_2024_a_2027, index=años, columns=['Pronóstico'])
#print(pronosticos_df)

c:\Users\ninic\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   19
Model:               SARIMAX(0, 1, 0)   Log Likelihood                -171.852
Date:                Tue, 26 Mar 2024   AIC                            347.704
Time:                        19:25:02   BIC                            349.485
Sample:                    01-01-2005   HQIC                           347.950
                         - 01-01-2023                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   2941.9728   1309.208      2.247      0.025     375.972    5507.974
sigma2      1.149e+07   3.21e+06      3.581      0.000     5.2e+06    1.78e+07
Ljung-Box (L1) (Q):                   0.94   Jarque-